In [ ]:
import pandas as pd
from transformers import BertTokenizer
import torch

In [ ]:
"""dictionary"""
DICT_PATH = "./dataset/TC/Lexicon2003-72k.txt"
df_dict = pd.read_csv(DICT_PATH, encoding="cp950", header=None)

In [ ]:
Human_N = '1'
Percent = {10:'01', 20:'02', 30:'03'}
FILE_NAME = "PTSND20011107_1"
ANS_PATH = "./dataset/TC/TestFile_TD_DOS/DocRatio/Human_" + Human_N + "/" + Percent[10] + "/" + FILE_NAME + ".txt"
ans_file = open(ANS_PATH)
raw_ans = ans_file.read()

In [ ]:
raw_ans = raw_ans.replace("a","").replace("\n", "")
ans = raw_ans.split(" ")
ans = ans[:-1]

In [ ]:
"""
answer decoder
input: index sequence (list of string)
output: decode of index (string)
"""
def decoder(index):
    result = ""
    for i in index:
        result += df_dict.loc[int(i), 0]
#     print("f",result)
    return result
ans = decoder(ans)
# ans

In [ ]:
NEWS_PATH = "./dataset/TC/TestFile_TD_DOS/" + FILE_NAME + ".txt"
news_file = open(NEWS_PATH, encoding="cp950")
news = news_file.read()

In [ ]:
"""
content preprocess
input: content (string), answer (string)
output: processed content(string), label (list of 1/0), cls id (list of int),
        origin content (list of string), target text (list of string)
"""
def content_preprocess(content, ans):
    content = content.replace(" ", "")
    content_list = content.splitlines()
    origin_content = content_list[:] # create a shallow copy, if don't add [:] will get the pointer
    result_label = []
    segments_ids = []
    cls_ids = []
    tgt_text_list = []
    pos = 0
    for i, text in enumerate(content_list):
        if (pos < 512):
            cls_ids.append(pos)
            pos += len(text) + 2 # +2 for [CLS], [SEP]
            if (ans.find(text) != -1):
                result_label.append(1) # label
                tgt_text_list.append(text)
            else:
                result_label.append(0) # label
        content_list[i] = "[CLS] " + text + " [SEP]"
    result_content = ' '.join(content_list)
    tgt_text = '<q>'.join(tgt_text_list)
    return result_content, result_label, cls_ids, origin_content, tgt_text
content, label, cls_ids, origin_content, tgt_text = content_preprocess(news, ans)
print(label)
print(cls_ids)
print(origin_content)
print(tgt_text)
content

In [ ]:
print(len(label))
print(len(cls_ids))

In [ ]:
from transformers import BertTokenizer
LM = "hfl/chinese-bert-wwm"
tokenizer = BertTokenizer.from_pretrained(LM)



In [ ]:
bert_dict = tokenizer.encode_plus(content,
                                         add_special_tokens = False,
                                         return_token_type_ids = False,
                                         max_length=512,
                                         pad_to_max_length=True,
                                         return_tensors='pt',
                                         truncation=True)
print(bert_dict)

In [ ]:
"""
segement embedding
code from BERTSUM
input: input_ids (list of int)
output: segment embedding (list of int)
"""
def create_segment(input_ids):
    segments_ids = []
    _segs = [-1] + [i for i, t in enumerate(input_ids) if t == tokenizer.vocab['[SEP]']]
    segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
    for i, s in enumerate(segs):
        if (i % 2 == 0):
            segments_ids += s* [0]
        else:
            segments_ids += s * [1]
    return segments_ids
bert_dict['input_ids'][0][511] = tokenizer.vocab['[SEP]']
segments_ids = create_segment(bert_dict['input_ids'][0])
# print(len(bert_dict['input_ids'][0]))
# print(len(segments_ids))

In [ ]:
data_dict = {"src": bert_dict['input_ids'][0].tolist(), "labels": label, "segs": segments_ids, 'clss': cls_ids,
                       'src_txt': origin_content, "tgt_txt": tgt_text}
# print(data_dict)

In [ ]:
import torch
dataset = []
dataset.append(data_dict)
torch.save(dataset, "simple_test.train.0.bert.pt")

In [8]:
import torch
# temp = torch.load("./dataset/bert_data_for_BERTSUM/TC/PTS.test.all.bert.pt")
# temp = torch.load("../BertSum/bert_data/NewsSummary/PTS.test.0.bert.pt")
temp = torch.load("../BertSum/bert_data/cnndm.test.0.bert.pt")
print(temp[0])

{'src': [101, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 2886, 1999, 4199, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 2018, 2069, 2074, 3369, 2005, 1037, 13609, 2565, 1999, 3304, 2043, 1996, 5043, 3047, 1999, 2254, 1012, 102, 101, 2002, 2001, 10583, 2067, 2000, 3190, 3081, 2250, 10771, 2006, 2233, 2322, 1010, 2021, 2002, 2351, 2006, 4465, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 102, 101, 2002, 2001, 2579, 2000, 1037, 2966, 4322, 1999, 1996, 3190, 2181, 1010, 2485, 2000, 2010, 2155, 2188, 1999, 8904, 3449, 9644, 1012, 102, 101, 2002, 2351, 2006, 4465, 2012, 7855, 3986, 2902, 1011, 2966, 19684, 1005, 1055, 2436, 14056, 3581, 18454, 6199, 2319, 2758, 1037, 3426, 1997, 2331, 24185, 1050, 1005, 1056, 2

In [7]:
print(len(temp))

20


---

In [ ]:
import torch
temp = torch.load("../BertSum/bert_data/cnndm.train.0.bert.pt")
# temp = torch.load("./dataset/bert_data_for_BERTSUM/TC/PTS.train.0.bert.pt")
# temp = torch.load("./simple_test.train.0.bert.pt")

In [ ]:
print(temp[0])

In [ ]:
r = set()
for i, e in enumerate(temp):
    if (len(e['src']) != 512):
        print(i, " src != 512")
        r.add(i)
    elif (len(e['labels']) != len(e['clss'])):
        print(i, " labels != clss")
        r.add(i)
    elif (len(e['segs']) != 512):
        print(i, " segs != 512")
        r.add(i)
r

In [ ]:
print(len(temp[0]['clss']))

---
## LCSTS

In [ ]:
with open('./dataset/LCSTS2.0/DATA/PART_II.txt', 'r') as rf:
    with open('./dataset/LCSTS2.0/DATA/PART_II.xml', 'w') as wf:
        text = rf.readline()
        i = 0
        while (text != ""):
            if (text.find("<doc id=") != -1):
                wf.write("<doc id=\"" + str(i) + "\">\n")
                i += 1
            else:
                wf.write(text)
            text = rf.readline()

In [ ]:
import xml.etree.cElementTree as ET
tree = ET.parse('./dataset/LCSTS2.0/DATA/PART_II.xml')
# tree = ET.parse('./dataset/TC/TestFile_TD_DOS/GoldHair_DocRatio_REF_20Test_Text01.xml')
root = tree.getroot()

In [ ]:
df_dict = {'summary' : [], 'text' : []}
for child in root:
#     print(child.tag, "-", child.attrib)
#     print(child[1].text)
#     print(child[2].text)
    df_dict['summary'].append(child[1].text.replace('\n', '').replace(" ", ""))
    df_dict['text'].append(child[2].text.replace('\n', '').replace(" ", ""))
df = pd.DataFrame(data=df_dict)
df.to_csv("./dataset/LCSTS2.0/DATA/PART_II.csv", index=False)

In [ ]:
df = pd.read_csv("./dataset/LCSTS2.0/DATA/PART_II.csv")
df

In [ ]:
import torch
def _pad(data, pad_id, width=-1):
        if (width == -1):
            width = max(len(d) for d in data)
        rtn_data = [d + [pad_id] * (width - len(d)) for d in data]
        return rtn_data
pre_clss = [[0, 27, 82, 135, 156, 185, 193, 249, 266, 288, 316, 357, 372, 428, 449, 476],
           [0, 16, 28, 65, 101, 125, 154, 194, 235, 282, 306, 339, 382, 425, 445, 483, 510]]
clss = torch.tensor(_pad(pre_clss, -1))
mask_cls = ~ (clss == -1)
clss[clss == -1] = 0
print(mask_cls)
print(clss)